In [ ]:
def add_link_id_to_tntp(file_path, output_file_path):
    # Open the file to read
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Prepare to write the new file
    with open(output_file_path, 'w') as f_out:
        link_id = 0
        is_data_section = False
        
        for line in lines:
            # Check if the line is the header or metadata
            if line.startswith("~") and not is_data_section:
                # Modify the header to add "link_id" after "~"
                new_header = "~\tlink_id\tinit_node\tterm_node\tcapacity\tlength\tfree_flow_time\tb\tpower\tspeed\ttoll\tlink_type\t;\n"
                f_out.write(new_header)
            # After the END OF METADATA, start adding link_id to each row
            elif is_data_section:
                if line.strip():
                    parts = line.strip().split("\t")
                    # Insert link_id as the first element and write the modified line
                    new_line = f"\t{link_id}\t" + "\t".join(parts[1:])
                    f_out.write(new_line + "\n")
                    link_id += 1
            else:
                # Check if the line marks the end of metadata
                if "<END OF METADATA>" in line:
                    f_out.write(line)
                    is_data_section = True
                else:
                    f_out.write(line)

def update_link_id(file_path, output_file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    with open(output_file_path, 'w') as f_out:
        is_data_section = False
        link_id = 0  # Start the link_id from 0
        
        for line in lines:
            if line.startswith("~") and not is_data_section:
                # Write the header as it is
                f_out.write(line)
            elif is_data_section:
                if line.strip():
                    parts = line.strip().split("\t")
                    # Modify link_id and write the line
                    parts[1] = str(link_id)  # Update the link_id to start from 0
                    f_out.write("\t".join(parts) + "\n")
                    link_id += 1
            else:
                # Check if the line marks the end of metadata
                if "<END OF METADATA>" in line:
                    f_out.write(line)
                    is_data_section = True
                else:
                    # Write metadata lines as they are
                    f_out.write(line)

# Usage example:
input_file_path = "../Generate_data/Bermingham/Birmingham_Net1.tntp"   # Input file path
output_file_path = "../Generate_data/Bermingham/Birmingham_Net2.tntp" # Output file path

update_link_id(input_file_path, output_file_path)


In [14]:
import pickle
import random
%run utils.py

num_nodes = 74
min_demand = 100
max_demand = 2000
num_pairs = int((num_nodes**2)*0.7)
# num_pairs = 1000
num_matrix = 50
filename = f'../Generate_data/EMA/od_demand_{num_matrix}.pkl'

# Create 1000 OD demand
od_demand_list = [generate_OD_demand(num_nodes, min_demand, max_demand, num_pairs) for i in range(num_matrix)]
with open(filename, 'wb') as f:
    pickle.dump(od_demand_list, f)

a_file = open(filename, "rb")
output = pickle.load(a_file)
a_file.close()

for i, od_demand in enumerate(output[:2]):
    print(len(od_demand))
    print(od_demand)

3833
{(59, 55): 732, (18, 17): 174, (70, 64): 1487, (71, 29): 1688, (50, 6): 1993, (42, 2): 1933, (28, 67): 326, (29, 32): 164, (8, 9): 872, (48, 45): 941, (40, 41): 789, (19, 18): 833, (60, 19): 811, (6, 48): 1133, (11, 14): 445, (52, 15): 819, (29, 68): 1916, (21, 64): 733, (41, 42): 940, (73, 74): 211, (33, 38): 1494, (52, 51): 1075, (53, 16): 605, (72, 29): 1707, (45, 12): 1027, (37, 8): 384, (10, 54): 1095, (56, 21): 1483, (2, 50): 767, (3, 15): 1675, (22, 28): 1324, (67, 36): 758, (13, 65): 1682, (59, 32): 1332, (46, 13): 419, (38, 9): 1935, (70, 41): 464, (49, 18): 340, (36, 48): 191, (55, 61): 443, (47, 57): 1136, (48, 22): 1882, (7, 21): 522, (18, 30): 1360, (59, 68): 1310, (5, 60): 944, (30, 4): 954, (71, 42): 826, (63, 38): 580, (9, 67): 236, (29, 45): 557, (48, 58): 1776, (21, 41): 1893, (41, 19): 1037, (33, 15): 1592, (6, 61): 1002, (52, 28): 105, (25, 11): 1469, (72, 6): 1977, (64, 2): 1625, (10, 31): 1760, (2, 27): 610, (22, 5): 383, (54, 74): 393, (34, 16): 1131, (74, 5

In [1]:
# from utils import *
from tqdm.notebook import tqdm
%run utils.py

a_file = open('../Generate_data/SiouxFalls/od_demand_1.pkl', "rb")
stats = pickle.load(a_file)
a_file.close()

Network, Nodes, links, cap, fft, alpha, beta, lengths = readNet('../Generate_data/SiouxFalls/SiouxFalls_net.tntp')

time = 0

for OD_matrix in tqdm(stats[:1]) :
    if time < 4000:
        print(time)
        data, Q, OD, O_D,n = get_data(Network, Nodes, links, cap, fft, alpha, beta, lengths, OD_matrix )
        flows, li, status, sol_time = TA_UE(data, n, OD, Q)
        if status ==2 :
            dataa = {'data' : data, 'path_flow' : flows, 'link_flow' : li, 'model_status': status, 'solving_time':sol_time}
            file_data = open("../Solution/SiouxFalls/MultiClass/UEdata_{}".format(time+1), "wb")
            pickle.dump(dataa , file_data)
            file_data.close()
            time += 1
        else : 
            print("Error ! Networks details :")
            print(OD_matrix)
            print("*****************************")

  0%|          | 0/1 [00:00<?, ?it/s]

0
Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-24


: 

In [4]:
# Add free flow travel time for truck 
net_file = 'SiouxFalls/SiouxFalls_net.tntp'
with open(net_file, 'r') as file:
    lines = file.readlines()

start_idx = next(i for i, line in enumerate(lines) if '~' in line and 'link_id' in line) + 1
data = pd.read_csv(net_file, sep='\t', skiprows=start_idx, header=None)
data.columns = ['~', 'link_id', 'init_node', 'term_node', 'capacity', 'length', 
                'free_flow_time', 'b', 'power', 'speed', 'toll', 'link_type', ';']
data['fft'] = data['free_flow_time'].apply(lambda x: (x,random.randint(x, x + 2)))

print(data)
output_path = 'SiouxFalls/SiouxFalls_net_multiClass.tntp'
with open(output_path, 'w') as file:
    file.writelines(lines[:start_idx-1])
    data.to_csv(file, sep='\t', index=False, header=True)

     ~  link_id  init_node  term_node      capacity  length  free_flow_time  \
0  NaN        0          1          2  25900.200640       6               6   
1  NaN        1          1          3  23403.473190       4               4   
2  NaN        2          2          1  25900.200640       6               6   
3  NaN        3          2          6   4958.180928       5               5   
4  NaN        4          3          1  23403.473190       4               4   
..  ..      ...        ...        ...           ...     ...             ...   
71 NaN       71         23         22   5000.000000       4               4   
72 NaN       72         23         24   5078.508436       2               2   
73 NaN       73         24         13   5091.256152       4               4   
74 NaN       74         24         21   4885.357564       3               3   
75 NaN       75         24         23   5078.508436       2               2   

       b  power  speed  toll  link_type  ;     fft 

### SOLVE UE FOR LINK REMOVED NETWORK

In [19]:
%run utils.py

# Get full feasible path set for full network
demand_file = 'EMA/od_demand_50.pkl'
net_file = 'EMA/EMA_net2.tntp'
path_set_dict, pair_path = get_full_paths(demand_file, net_file, path_num=3)

with open('EMA/unique_paths.pkl', 'wb') as file:
    pickle.dump(path_set_dict, file)

  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
print(len(path_set_dict))
print((74*74-74)*3)

16122
16206


In [47]:
%run utils.py

demand_file = 'SiouxFalls/od_demand_4000.pkl'
net_file = 'SiouxFalls/SiouxFalls_net_multiClass.tntp'
output_file = '../Solution/SiouxFalls/MultiClass/output'
to_solve=4000

solve_UE(net_file, demand_file, pair_path, output_file, to_solve)

# stat = read_file(demand_file)
# Network, Nodes, links, cap, fft, alpha, beta, lengths = readNet(net_file)

# time = 0
# for OD_matrix in tqdm(stat):
#         paths = {k: (pair_path[k][:3] if len(pair_path[k]) >= 3 else pair_path[k]) for k in OD_matrix.keys()}
#         data, Q, OD, O_D,n = get_data_N(Network, Nodes, links, cap, fft, alpha, beta, lengths, OD_matrix, paths)
#         flows, linkss = TA_UE(data, n, OD, Q)
#         # if status ==2 :
#         dataa = {'data' : data, 'path_flow' : flows, 'link_flow' : linkss}
#         file_data = open(output_file+str(time), "wb")
#         pickle.dump(dataa , file_data)
#         file_data.close()

  0%|          | 0/4000 [00:00<?, ?it/s]

0
model solved in: 2.86694598197937
1
model solved in: 2.594627618789673
2
model solved in: 2.2644307613372803
3
model solved in: 2.6288769245147705
4
model solved in: 1.9680449962615967
5
model solved in: 2.1747660636901855
6
model solved in: 2.02964186668396
7
model solved in: 1.9715492725372314
8
model solved in: 1.8858449459075928
9
model solved in: 2.1215670108795166
10
model solved in: 2.0521841049194336
11
model solved in: 1.9804751873016357
12
model solved in: 1.9384441375732422
13
model solved in: 2.0564889907836914
14
model solved in: 1.812927007675171
15
model solved in: 1.7421197891235352
16
model solved in: 1.683372974395752
17
model solved in: 1.762192964553833
18
model solved in: 2.0079071521759033
19
model solved in: 1.777919054031372
20
model solved in: 1.7266347408294678
21
model solved in: 1.8739120960235596
22
model solved in: 1.7768011093139648
23
model solved in: 1.8601269721984863
24
model solved in: 1.6896650791168213
25
model solved in: 1.9693610668182373
26
mo

In [15]:
%run utils.py

# Remove links from origin network
new_net_file = 'SiouxFalls/SiouxFalls_net_remove2.tntp'
remove_ids = [5, 20]
remove_links_from_tntp(net_file, new_net_file, remove_ids)
new_pair_path = remove_links_from_path(pair_path, remove_ids)

output_file = '../Solution/SiouxFalls/Remove2/SiouxFall_UE_'
solve_UE(new_net_file, demand_file, new_pair_path, output_file, to_solve=400)


  0%|          | 0/400 [00:00<?, ?it/s]

model solved in: 1.2197239398956299
Finish solving file  1
model solved in: 1.0642778873443604
Finish solving file  2
model solved in: 1.3703291416168213
Finish solving file  3
model solved in: 1.2543740272521973
Finish solving file  4
model solved in: 1.2483389377593994
Finish solving file  5
model solved in: 1.7124059200286865
Finish solving file  6
model solved in: 1.179673671722412
Finish solving file  7
model solved in: 1.1862890720367432
Finish solving file  8
model solved in: 1.0997717380523682
Finish solving file  9
model solved in: 1.0245490074157715
Finish solving file  10
model solved in: 0.8811361789703369
Finish solving file  11
model solved in: 0.8293969631195068
Finish solving file  12
model solved in: 0.811682939529419
Finish solving file  13
model solved in: 1.0440361499786377
Finish solving file  14
model solved in: 0.9164431095123291
Finish solving file  15
model solved in: 1.2894129753112793
Finish solving file  16
model solved in: 1.0790419578552246
Finish solving 

# Remove 3 links from origin network


In [ ]:
%run utils.py

new_net_file = 'SiouxFalls/SiouxFalls_net_remove3.tntp'
remove_ids = [5, 20, 40]
remove_links_from_tntp(net_file, new_net_file, remove_ids)
new_pair_path = remove_links_from_path(pair_path, remove_ids)

output_file = '../Solution/SiouxFalls/Remove3/SiouxFall_UE_'
solve_UE(new_net_file, demand_file, new_pair_path, output_file, to_solve=400)